In [81]:
import pandas as pd
import numpy as np
from datetime import datetime

def convert_to_extension_dtypes(data):
    """
    Convert the data types of a given DataFrame or Series to Pandas extension data types.
    
    :param data: DataFrame or Series
    :return: DataFrame or Series with updated data types
    """
    # Check if the input is a DataFrame or Series
    if not isinstance(data, (pd.DataFrame, pd.Series)):
        raise ValueError("Input must be a pandas DataFrame or Series")

    # Convert data types for DataFrame
    if isinstance(data, pd.DataFrame):
        for col in data.columns:
            col_data = data[col]
            # Convert to appropriate extension dtype
            # if pd.api.types.is_integer_dtype(col_data):
            #     data[col] = col_data.astype("Int64")
            # elif pd.api.types.is_float_dtype(col_data):
            #     data[col] = col_data.astype("Float64")
            
            if pd.api.types.is_string_dtype(col_data):
                data[col] = col_data.astype("string")
            elif pd.api.types.is_bool_dtype(col_data):
                data[col] = col_data.astype("boolean")

    # Convert data type for Series
    else:
        if pd.api.types.is_integer_dtype(data):
            data = data.astype("Int64")
        elif pd.api.types.is_float_dtype(data):
            data = data.astype("Float64")
        elif pd.api.types.is_string_dtype(data):
            data = data.astype("string")
        elif pd.api.types.is_bool_dtype(data):
            data = data.astype("boolean")

    return data
